In [1]:
%matplotlib inline
import matplotlib
from topic_coherence_experiments import *

def rerank(dataset, param_ids, nbs_topics, topns=None, metrics=None, evaluate=False, save=True):
    if topns is None:
        topns = [20]
    if metrics is None:
        metrics = ['u_mass', 'c_v', 'c_uci', 'c_npmi', 'vote']

    reranker = None
    all_topic_candidates = []
    for n in topns:
        topics_loader = TopicsLoader(dataset, param_ids, nbs_topics, topn=n)
        reranker = Reranker(topics_loader, nb_candidate_terms=n, processes=4)
        reranker.rerank_fast(metrics)
        if save:
            reranker.save_results(join(ETL_PATH, 'Reranker'))
        all_topic_candidates.append(reranker)

    all_topic_candidates = pd.concat(all_topic_candidates)

    if evaluate:
        all_tc_list = all_topic_candidates.drop('metric', axis=1).values.tolist()
        scores = reranker.evaluate(all_tc_list)

        columns = ['ref'] + metrics
        columns = ['_'.join([s, str(n)]) for n in topns for s in columns]
        stats_umass = reranker.stats(scores['u_mass'], columns).assign(eval_metric='u_mass')
        stats_cv = reranker.stats(scores['c_v'], columns).assign(eval_metric='c_v')
        stats_cuci = reranker.stats(scores['c_uci'], columns).assign(eval_metric='c_uci')
        stats_cnpmi = reranker.stats(scores['c_npmi'], columns).assign(eval_metric='c_npmi')
        stats = (
            pd.concat([stats_umass, stats_cv, stats_cuci, stats_cnpmi])
            .assign(
                dataset=dataset,
                # nbtopics=nbtopics,
                # param=param
            )
        )
    else:
        stats = None

    return all_topic_candidates, stats
    
param_ids = ['a42', 'b42', 'c42', 'd42', 'e42']
nbs_topics = [10, 25, 50, 100]

candidates, stats = rerank(datasets['O'], param_ids[:2], nbs_topics[:2])
candidates
#stats = evaluate(topics)
#tl = TopicsLoader(datasets['O'], param_ids=param_ids, nbs_topics=nbs_topics)
#tl.topics

loading dictionary from ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_bow.dict
loading corpus from ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_bow.mm
loading texts from ../data/preprocessed/LDAmodel/OnlineParticipation_fullset_nouns_texts.json
loading model from ../data/preprocessed/LDAmodel/a42/OnlineParticipation_LDAmodel_a42_10
loading model from ../data/preprocessed/LDAmodel/b42/OnlineParticipation_LDAmodel_b42_10
creating reranked top candidates for metrics ['u_mass', 'c_v', 'c_uci', 'c_npmi', 'vote'], using fast method
producing coherence scores for u_mass measure using 20 candidate terms for 20 topics
avg out of place score compared to original topic term rankings: 40.7
done in 00:00:01
producing coherence scores for c_v measure using 20 candidate terms for 20 topics
avg out of place score compared to original topic term rankings: 29.1
done in 00:00:09
producing coherence scores for c_uci measure using 20 candidate terms for 20 topics
avg ou

KeyError: 'nb_topics'